In [53]:
# python3 -m venv env
# source env/bin/activate
# Install below packages in the virtual env created
# pip3 install langgraph-supervisor==0.0.4 
# pip3 install langchain-openai==0.3.6 
# pip3 install langgraph==0.2.74 
# pip3 install langgraph-sdk==0.1.51
# pip3 install langchain==0.3.19
# pip3 install langgraph-prebuilt==0.1.1
# pip3 install langgraph-checkpoint==2.0.16

SyntaxError: invalid syntax (1975131222.py, line 1)

In [ ]:
import os

# Set the environment variables with the Azure OpenAI deployment details
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"]= ""
os.environ["AZURE_OPENAI_API_KEY"] = ""

In [13]:
os.environ["JIRA_URL"] = ""
os.environ["JIRA_USERNAME"] = ""
os.environ["JIRA_API_TOKEN"] = ""

In [20]:

import sys
import requests
import json
import uuid
from typing import TypedDict, Annotated, List, Literal

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

from langchain_openai import AzureChatOpenAI

from pydantic import BaseModel

from IPython.display import Image, display

model = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # Replace with your deployment name
    api_version="2024-08-01-preview",  # Use the correct API version
    temperature=0
)

supported_tools = ["create_jira_project", 
                    "get_jira_project", 
                    "get_jira_issue_by_project_name",
                    "create_jira_issue"]

# class State(TypedDict, total=False):
#     messages: Annotated[List[BaseMessage], add_messages]
# 
# state: State = State()
# 
# def get_state():
#     return state

# Final output to caller
class JiraOutput(BaseModel):
    """
    The output.
        Attributes:
        response (str): detail response.
    """
    response: str

def get_tool_output(result, supported_tools) -> JiraOutput:

    try:
        # Check if 'messages' key exists in the result
        response_to_send_list: List[str] = []
        if 'messages' in result:
            for message in result['messages']:
                if message:
                    if hasattr(message, 'tool_call_id'):
                        if hasattr(message, 'name') and message.name in supported_tools:
                            if hasattr(message, 'content'):
                                print(f"\nTool: {message.name}, Content: {message.content}")
                                # message.content is a string, ideally we would want the structured_json here
                                # parse the content to get the required output
                                output = message.content 
                                token = "="
                                parts = output.partition(token)
                                if len(parts) == 3:
                                    output_after_token = output.partition(token)[2].strip('\'')
                                    # append results for all tools
                                    if output_after_token:
                                        response_to_send_list.append(output_after_token)
        else:
            print("Error: 'messages' not found in the result")
            return JiraOutput(response="error performing the operation")
        
        # cumulative response from all tools
        if len(response_to_send_list) == 0:
            return JiraOutput(response="")
        
        if len(response_to_send_list) > 1:
            response_to_send = ",".join(response_to_send_list)
        else:
            response_to_send = response_to_send_list[0]
            
        return JiraOutput(response=response_to_send)
    
    except Exception as e:
        tb = sys.exc_info()[2]
        # Extract the line number from the traceback
        line_number = tb.tb_lineno
        print(f"Output Exception: {e}, lineno: {line_number}")
        return JiraOutput(response="error performing the operation")

# Create specialized agents

# this model should be generic for all responses to be sent from Jira issues agent
# for each tool handled by this agent
class JiraIssueOutput(BaseModel):
    """
    The output.
        Attributes:
        response (str): detail response from tool.
    """
    # status_code: int
    response: str
    
class CreateJiraIssueInput(BaseModel):
    """
    The input for creating a Jira issues.
        Attributes:
        op_type : Literal["create_jira_issue"]
        project_key (str): The key of the project.
        summary (str): The summary of the issue.
        description (str): The description of the issue.
        issue_type (str): The type of the issue.
        reporter_email(str): The email of the reporter.
    """
    op_type: Literal["create_jira_issue"]
    project_key: str
    summary: str
    description: str
    issue_type: str
    reporter_email: str
    
# Agent 1 (Jira Issues)
def create_jira_issue(input: CreateJiraIssueInput) -> JiraIssueOutput:
    """create a jira issue and return the output.
         Args:
         input (CreateJiraIssueInput):
             The user-provided input that guides the jira issue creation. 
             This request is serialized from a `CreateJiraIssueInput` object,
             which must have a `model_dump()` method for JSON conversion.
         
     Returns:
         JiraIssueOutput:
             A JSON representation of the JiraIssueOutput.
             This response is serialized from a `JiraIssueOutput` object,
             which must have a `model_dump()` method for JSON conversion.
    """
    print("\ncreate_jira_issue - Received input:", input)
    if not input or input is None:
        return JiraIssueOutput(response="")
    
    response_str = f"Created Jira issue for project: {input.project_key} with id: xxxx"
    return JiraIssueOutput(response=response_str)


class GetJiraIssueByProjectNameInput(BaseModel):
    """
    The input for getting a Jira issue by project name.
        Attributes:
        op_type : Literal["get_jira_issue_by_project_name"]
        project_name (str): The name of the project.
    """
    op_type: Literal["get_jira_issue_by_project_name"]
    project_name: str
    
# Agent 1 (Jira Issues)
def get_jira_issue_by_project_name(input: GetJiraIssueByProjectNameInput) -> JiraIssueOutput:
    """get a jira issue and return the output.
         Args:
         input (GetJiraIssueByProjectNameInput):
             The user-provided input that guides the jira issue retrieval. 
             This request is serialized from a `GetJiraIssueByProjectNameInput` object,
             which must have a `model_dump()` method for JSON conversion.
         
     Returns:
         JiraIssueOutput:
             A JSON representation of the JiraIssueOutput.
             This response is serialized from a `JiraIssueOutput` object,
             which must have a `model_dump()` method for JSON conversion.
    """
    print("\nget_jira_issue_by_project_name - Received input:", input)
    if not input or input is None:
        return JiraIssueOutput(response="error performing the operation")
    
    response_str = f"Got Jira issue for project: {input.project_name} with id: 5678"
    return JiraIssueOutput(response=response_str)

class GetIssueMetadataInput(BaseModel):
    """
    The input for getting metadata of a Jira issue.
        Attributes:
        op_type : Literal["get_issue_metadata"]
        project_key (str): The key of the project.
        issue_type (str): The type of the issue.
    """
    op_type: Literal["get_issue_metadata"]
    project_key: str
    issue_type: str


def get_issue_metadata(input: GetIssueMetadataInput) -> JiraIssueOutput:
    """Get metadata of a Jira issue and return the output.
         Args:
         input (GetIssueMetadataInput):
             The user-provided input that guides the issue metadata retrieval. 
             This request is serialized from a `GetIssueMetadataInput` object,
             which must have a `model_dump()` method for JSON conversion.
         
     Returns:
         IssueMetadataOutput:
             A JSON representation of the IssueMetadataOutput.
             This response is serialized from a `IssueMetadataOutput` object,
             which must have a `model_dump()` method for JSON conversion.
    """
    print("\nget_issue_metadata - Received input:", input)
    if not input or input is None:
        return IssueMetadataOutput(metadata={})
    
    # Replace with your Jira instance URL and authentication details
    jira_url = os.getenv("JIRA_URL")
    auth = (os.getenv("JIRA_USERNAME"), os.getenv("JIRA_API_TOKEN"))
    
    # Check if Jira environment details are missing
    if auth[0] is None or auth[1] is None or jira_url is None:
        # Return fake data
        print("Jira environment details are missing")
        fake_metadata = {
            "projectKeys": input.project_key,
            "issuetypeNames": input.issue_type,
            "fields": {
                "summary": {"required": True, "type": "string"},
                "description": {"required": False, "type": "string"},
                "priority": {"required": False, "type": "string"}
            }
        }
        return IssueMetadataOutput(metadata=fake_metadata)
    
    
    # Construct the URL for the metadata endpoint
    url = f"{jira_url}/rest/api/3/issue/createmeta?projectKeys={input.project_key}&issuetypeNames={input.issue_type}&expand=projects.issuetypes.fields"
    
    # Make the request to the Jira API
    response = requests.get(url, auth=auth)
    
    if response.status_code != 200:
        return IssueMetadataOutput(metadata={"error": "Failed to fetch metadata"})
    
    metadata = response.json()
    response_str = f"Got metadata for project: {input.project_key} and issue type: {input.issue_type}, metadata: {metadata}"
    return JiraIssueOutput(response=response_str)

# Add this tool to your jira_issues_agent
jira_issues_agent = create_react_agent(
    model=model,
    tools=[get_issue_metadata, create_jira_issue, get_jira_issue_by_project_name],
    name="jira_issues_agent",
    prompt=(
        "You are a helpful agent. Only use the tools available.\n"
        "1. You can only handle issues.\n"
        "2. Before creating an issue, always call the `get_issue_metadata` tool to retrieve the necessary metadata. Only create issues that are supported by the tool.\n"
    ),
    response_format=JiraIssueOutput
)

# this model should be generic for all responses to be sent from Jira projects agent
# for each tool handled by this agent
class JiraProjectOutput(BaseModel):
    """
    The output.
        Attributes:
        response (str): detail response.
    """
    # status_code: int
    response: str

class CreateJiraProjectInput(BaseModel):
    """
    The input for creating a Jira project.
        Attributes:
        op_type : Literal["create_jira_project"]
        name (str): The name of the project.
    """
    op_type: Literal["create_jira_project"]
    name: str

# Agent 2 (Jira Projects)
def create_jira_project(input: CreateJiraProjectInput) -> JiraProjectOutput:
    """create a jira project and return the output.
         Args:
         input (CreateJiraProjectInput):
             The user-provided input that guides the jira project creation. 
             This request is serialized from a `CreateJiraProjectInput` object,
             which must have a `model_dump()` method for JSON conversion.
         
     Returns:
         JiraProjectOutput:
             A JSON representation of the JiraProjectOutput.
             This response is serialized from a `JiraProjectOutput` object,
             which must have a `model_dump()` method for JSON conversion.
    """
    print("\ncreate_jira_project - Received input:", input)
    if not input or input is None:
        return JiraProjectOutput(response="error performing the operation")
    
    response_str = f"Created Jira project: {input.name} with id: 1234"
    return JiraProjectOutput(response=response_str)

class GetJiraProjectInput(BaseModel):
    """
    The input for get on a Jira project.
        Attributes:
        op_type : Literal["get_jira_project"]
        name (str): The name of the project.
    """
    op_type: Literal["get_jira_project"]
    name: str

# Agent 2 (Jira Projects)
def get_jira_project(input: GetJiraProjectInput) -> JiraProjectOutput:
    """get a jira project and return the output.
         Args:
         input (GetJiraProjectInput):
             The user-provided input that guides the jira project retrieval. 
             This request is serialized from a `GetJiraProjectInput` object,
             which must have a `model_dump()` method for JSON conversion.
         
     Returns:
         JiraProjectOutput:
             A JSON representation of the JiraProjectOutput.
             This response is serialized from a `JiraProjectOutput` object,
             which must have a `model_dump()` method for JSON conversion.
    """
    print("\nget_jira_project - Received input:", input)
    if not input or input is None:
        return JiraProjectOutput(response="error performing the operation")
    
    response_str = f"Got Jira project: {input.name} with id: 1234 and owner: John Doe"
    return JiraProjectOutput(response=response_str)


jira_projects_agent = create_react_agent(
    model=model,
    tools=[create_jira_project, get_jira_project],
    name="jira_projects_agent",
    # prompt="You are a helpful agent. Only use the tools available. You must not validate the inputs or generate your own responses. Only use the tools Tool Message response as your final response.",
    prompt="You are a helpful agent. Only use the tools available."
            "1. you can only handle projects\n"
            "2. **Projects**: A project is a collection of issues. Projects can be of different types such as software, business, etc.\n"
            "3. Epics and stories are not projects. They are issues under a project.",
    response_format=JiraProjectOutput
)

# Create supervisor workflow
workflow = create_supervisor(
    supervisor_name="jira_supervisor",
    tools=[get_issue_metadata],
    agents=[jira_projects_agent, jira_issues_agent],
    model=model,
    # prompt=(
    #     "You are a team supervisor managing a jira projects agent and a jira issues agent."
    #     "For any jira projects only use the jira_projects_agent."
    #     "For any jira issues only use the jira_issues_agent."
    #     "If an agent is not found, return error message to the caller."
    # ),
    prompt=(
        "You are a team supervisor managing the provided jira agents."
        "Only use the agents provided"
        "If an agent is not found, return error message to the caller."
        "Here is the hierarchy of Jira issue types and projects:\n"
        "1. **Projects**: A project is a collection of issues. Projects can be of different types such as software, business, etc.\n"
        "2. **Issue Types**: Issues are the building blocks of a project." 
        # "2. **Issue Types**: Issues are the building blocks of a project. They can be of various types as indicated by the get_issue_metadata tool. Please use this tool prior to managing issues.\n"
        # "Please use this hierarchy to understand and manage Jira issues and projects."
        ),
    add_handoff_back_messages=True,
    output_mode="full_history",
)

checkpointer = InMemorySaver()

try:
    
    # Compile graph - can be done once and reused
    app = workflow.compile(checkpointer=checkpointer)

    # View Graph
    # display(Image(app.get_graph().draw_mermaid_png()))

    # pass unique thread_id for each invoke
    # config = {"configurable": {"thread_id": uuid.uuid4()}}
    
    # SAMPLES USING user-input = prompt
    
    # single agent and tool
    # user_prompt = "create a project for my venture test-venture-1"
    # print(f"\nUser prompt: {user_prompt}")
    # result = app.invoke({
    #     "messages": [
    #         {
    #             "role": "user",
    #             "content": user_prompt
    #         }
    #     ],
    # }, {"configurable": {"thread_id": uuid.uuid4()}})
    # print("\n", get_tool_output(result, supported_tools).model_dump_json())
    #  
    # # across 2 tools under same agent
    # user_prompt = "1. create a project for my venture test-venture-2 2. get the project details for it"
    # print(f"\nUser prompt: {user_prompt}")
    # result = app.invoke({
    #     "messages": [
    #         {
    #             "role": "user",
    #             "content": user_prompt
    #         }
    #     ],
    # }, {"configurable": {"thread_id": uuid.uuid4()}})
    # print("\n", get_tool_output(result, supported_tools).model_dump_json())

    # across tools in 2 agents 
    # user_prompt = "create a project for my venture test-venture-1 and get issues for project test-venture-1"
    # user_prompt = "create a user"
    #user_prompt = "Create an Epic in APT project for upgrading all active RDS instances to latest postgres version to avoid extended AWS support costs and create a Story under this Epic (asked by user_email: sraradhy@cisco.com)"
    user_prompt = "create a issue of to write a api service assigned to user sushroff@cisco.com as type sushroff-custom-issue in the APT project"
    
    print(f"\nUser prompt: {user_prompt}")
    result = app.invoke({
        "messages": [
            {
                "role": "user",
                "content": user_prompt
            }
        ],
    }, {"configurable": {"thread_id": uuid.uuid4()}})
    print("\n", get_tool_output(result, supported_tools).model_dump_json())
    
    # SAMPLES (non-NLP) USING predefined user-input req body
    
    # working input format: create project
    # jira_input = CreateJiraProjectInput(op_type="create_jira_project", 
    #                                         name="Project 1")
    # invalid input examples
    # jira_input = CreateJiraProjectInput(op_type="create_issue", 
    #                                     name="Project 1") # invalid op_type
    # jira_input = CreateJiraProjectInput(op_type=CREATE_JIRA_PROJECT_TYPE) #missing field name
    
    # result = app.invoke({
    #     "messages": [
    #         {
    #             "role": "user",
    #             "content": jira_input.model_dump_json()
    #         }
    #     ],
    # }, {"configurable": {"thread_id": uuid.uuid4()}})
    # print(f"Structured result:\n", result["structured_response"])
    
    # working input format: get project
    # jira_input = GetJiraProjectInput(op_type="get_jira_project", 
    #                                     name="Project 1")
    # invalid input examples
    # jira_input = GetJiraProjectInput(op_type="get_issue", 
    #                                     name="Project 1") # invalid op_type
    # jira_input = GetJiraProjectInput(op_type=GET_JIRA_PROJECT_TYPE) #missing field name
    
    # result = app.invoke({
    #     "messages": [
    #         {
    #             "role": "user",
    #             "content": jira_input.model_dump_json()
    #         }
    #     ],
    # }, {"configurable": {"thread_id": uuid.uuid4()}})
    # print(f"Structured result:\n", result["structured_response"])
    
    for m in result["messages"]:
        m.pretty_print()

        
except Exception as e:
    print(f"Exception: {e}")
    # return appropriate response to callerf

    



User prompt: create a issue of to write a api service assigned to user sushroff@cisco.com as type sushroff-custom-issue in the APT project

get_issue_metadata - Received input: op_type='get_issue_metadata' project_key='APT' issue_type='sushroff-custom-issue'

create_jira_issue - Received input: op_type='create_jira_issue' project_key='APT' summary='Write an API service' description='Task to develop and implement a new API service.' issue_type='sushroff-custom-issue' reporter_email='sushroff@cisco.com'

Tool: create_jira_issue, Content: response='Created Jira issue for project: APT with id: xxxx'

 {"response":"Created Jira issue for project: APT with id: xxxx"}
================================ Human Message =================================

create a issue of to write a api service assigned to user sushroff@cisco.com as type sushroff-custom-issue in the APT project
================================== Ai Message ==================================
Name: jira_supervisor
Tool Calls:
  get_